In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [2]:
curr_dir = os.getcwd()

In [6]:
import os
os.makedirs(curr_dir+"/train_data/dogs")
os.makedirs(curr_dir+"/train_data/cats")

In [7]:
os.makedirs(curr_dir+"/test_data/dogs")
os.makedirs(curr_dir+"/test_data/cats")

In [5]:
os.listdir("test_data/dogs")

[]

In [4]:
import os, cv2, random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils

C:\Users\BDCOE-ML\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
TRAIN_DIR = curr_dir+'//train/'
TEST_DIR = curr_dir+'//test/'
train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)] # use this for full dataset
train_dogs =   [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'dog' in i]
train_cats =   [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'cat' in i]

In [9]:
%%time
import shutil
for i in train_dogs[:11000]:
    a = i.strip().split('/')
    shutil.copyfile(i, 'train_data/dogs/'+a[3])
for i in train_cats[:11000]:
    a = i.strip().split('/')
    shutil.copyfile(i, 'train_data/cats/'+a[3])

Wall time: 3min 31s


In [10]:
len(train_dogs)

12500

In [11]:
len(os.listdir("train_data/dogs"))

11000

In [12]:
for i in train_dogs[11000:12500]:
    a = i.strip().split('/')
    shutil.copyfile(i, 'test_data/dogs/'+a[3])
for i in train_cats[11000:12500]:
    a = i.strip().split('/')
    shutil.copyfile(i, 'test_data/cats/'+a[3])

In [ ]:
os.listdir("test_data/dogs")

In [13]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('train_data',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('test_data',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         steps_per_epoch = 22000,
                         epochs = 10,
                         validation_data = test_set,
                         validation_steps = 3000)

Found 22000 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.
Epoch 1/10
22000/22000 [==============================] - 3076s 140ms/step - loss: 0.3950 - acc: 0.8178 - val_loss: 0.3658 - val_acc: 0.8473
Epoch 2/10
22000/22000 [==============================] - 3075s 140ms/step - loss: 0.2793 - acc: 0.8808 - val_loss: 0.3708 - val_acc: 0.8577
Epoch 3/10
22000/22000 [==============================] - 3071s 140ms/step - loss: 0.2337 - acc: 0.9017 - val_loss: 0.3869 - val_acc: 0.8563
Epoch 4/10
22000/22000 [==============================] - 3073s 140ms/step - loss: 0.2056 - acc: 0.9140 - val_loss: 0.3878 - val_acc: 0.8619
Epoch 5/10
22000/22000 [==============================] - 3075s 140ms/step - loss: 0.1865 - acc: 0.9224 - val_loss: 0.3949 - val_acc: 0.8667
Epoch 6/10
22000/22000 [==============================] - 3073s 140ms/step - loss: 0.1729 - acc: 0.9282 - val_loss: 0.4295 - val_acc: 0.8611
Epoch 7/10
22000/22000 [==============================] - 3072s 140ms

In [14]:
classifier.save(curr_dir+'//my_model.h5')

In [15]:
classifier.save_weights(curr_dir+'//my_model_weights.h5')